# Load tfrecord

This small script helps to load the data from tfrecord train/test and to transform the audio wav from DCM format into numpy array.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import numpy as np
#tf.random.set_seed(42)

import os
import tensorflow.keras.backend as K
AUTO     = tf.data.experimental.AUTOTUNE

In [ ]:
def read_labeled_tfrecord(example, return_image_name=False):
    tfrec_format = {
        'recording_id'                        : tf.io.FixedLenFeature([], tf.string),
        'label_info'                   : tf.io.FixedLenFeature([], tf.string),
        'audio_wav'                       : tf.io.FixedLenFeature([], tf.string)
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    
    example["audio_wav"], example["sample_rate"] = tf.audio.decode_wav(example["audio_wav"])

    return example


def read_unlabeled_tfrecord(example):
    tfrec_format = {
        'recording_id'                        : tf.io.FixedLenFeature([], tf.string),
        'audio_wav'                       : tf.io.FixedLenFeature([], tf.string)
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    
    example["audio_wav"], example["sample_rate"] = tf.audio.decode_wav(example["audio_wav"])

    return example

In [ ]:
files_train = ["../input/rfcx-species-audio-detection/tfrecords/train/" + x for x in os.listdir("../input/rfcx-species-audio-detection/tfrecords/train")]
files_test = ["../input/rfcx-species-audio-detection/tfrecords/test/" + x for x in os.listdir("../input/rfcx-species-audio-detection/tfrecords/test")]
print("number of training tfrecord : ", len(files_train))
print("number of training tfrecord : ", len(files_test))

In [ ]:
samples_rates = set()
audio_lengths = set()
for path_tfrecord in files_train:
    raw_dataset = tf.data.TFRecordDataset(path_tfrecord).cache()
    raw_dataset = raw_dataset.map(lambda example: read_labeled_tfrecord(example), num_parallel_calls=AUTO)



    for raw_record in raw_dataset:
        samples_rates.add(raw_record["sample_rate"].numpy())
        audio_lengths.add(raw_record["audio_wav"].shape[0])
        
for path_tfrecord in files_test:
    raw_dataset = tf.data.TFRecordDataset(path_tfrecord).cache()
    raw_dataset = raw_dataset.map(lambda example: read_unlabeled_tfrecord(example), num_parallel_calls=AUTO)



    for raw_record in raw_dataset:
        samples_rates.add(raw_record["sample_rate"].numpy())
        audio_lengths.add(raw_record["audio_wav"].shape[0])

In [ ]:
print(samples_rates)
print(audio_lengths)

tfrecord have only samples with the same lengths and same samples rates. good.

In [ ]:
for path_tfrecord in files_train:
    raw_dataset = tf.data.TFRecordDataset(path_tfrecord).cache()
    raw_dataset = raw_dataset.map(lambda example: read_labeled_tfrecord(example), num_parallel_calls=AUTO)



    for i, raw_record in enumerate(raw_dataset):
        if i  > 10:
            break
        print(raw_record["label_info"].numpy().decode().strip())
        
    break